In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [2]:
dataDir='./coco'
dataType='merge2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

In [3]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=14.87s)
creating index...
index created!


In [4]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
for cat in cats:
    print(cat)

nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))
print('COCO categories lens: %d' % len(nms))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))
print('COCO sub-categories lens: %d' % len(nms))

{'supercategory': 'person', 'id': 1, 'name': 'person'}
{'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}
{'supercategory': 'vehicle', 'id': 3, 'name': 'car'}
{'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'}
{'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}
{'supercategory': 'vehicle', 'id': 6, 'name': 'bus'}
{'supercategory': 'vehicle', 'id': 7, 'name': 'train'}
{'supercategory': 'vehicle', 'id': 8, 'name': 'truck'}
{'supercategory': 'vehicle', 'id': 9, 'name': 'boat'}
{'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'}
{'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'}
{'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'}
{'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'}
{'supercategory': 'outdoor', 'id': 15, 'name': 'bench'}
{'supercategory': 'animal', 'id': 16, 'name': 'bird'}
{'supercategory': 'animal', 'id': 17, 'name': 'cat'}
{'supercategory': 'animal', 'id': 18, 'name': 'dog'}
{'supercategory': 'animal', 'id':

In [5]:
coco.info()
print(len(coco.getImgIds([])))
print(len(coco.imgs))
print(len(coco.anns))

description: COCO 2017 Dataset
url: http://cocodataset.org
version: 1.0
year: 2017
contributor: COCO Consortium
date_created: 2017/09/01
123287
123287
896782


In [6]:
import json

nms=[cat['name'] for cat in cats]
count = 0
object_net = {}
total=[]
for obj1 in nms:
    obj1_list = coco.getImgIds(catIds=coco.getCatIds(catNms=[obj1]));
    print("obj1 (%d): %s" % (len(obj1_list), obj1))
    total.extend(obj1_list)
    
    if obj1 not in object_net:
        object_net[obj1] = {}
        
    for obj2 in nms:
        if obj2 == obj1:
            object_net[obj1][obj2] = 1.0
            continue
            
        obj2_list = coco.getImgIds(catIds=coco.getCatIds(catNms=[obj2]));
        if obj2 not in object_net[obj1]:
            score = len(set(obj1_list).intersection(obj2_list)) *1.0 / len(set(obj1_list).union(obj2_list))
            
        object_net[obj1][obj2] = score


obj1 (66808): person
obj1 (3401): bicycle
obj1 (12786): car
obj1 (3661): motorcycle
obj1 (3083): airplane
obj1 (4141): bus
obj1 (3745): train
obj1 (6377): truck
obj1 (3146): boat
obj1 (4330): traffic light
obj1 (1797): fire hydrant
obj1 (1803): stop sign
obj1 (742): parking meter
obj1 (5805): bench
obj1 (3362): bird
obj1 (4298): cat
obj1 (4562): dog
obj1 (3069): horse
obj1 (1594): sheep
obj1 (2055): cow
obj1 (2232): elephant
obj1 (1009): bear
obj1 (2001): zebra
obj1 (2647): giraffe
obj1 (5756): backpack
obj1 (4142): umbrella
obj1 (7133): handbag
obj1 (3955): tie
obj1 (2507): suitcase
obj1 (2268): frisbee
obj1 (3202): skis
obj1 (1703): snowboard
obj1 (4431): sports ball
obj1 (2352): kite
obj1 (2603): baseball bat
obj1 (2729): baseball glove
obj1 (3603): skateboard
obj1 (3635): surfboard
obj1 (3561): tennis racket
obj1 (8880): bottle
obj1 (2643): wine glass
obj1 (9579): cup
obj1 (3710): fork
obj1 (4507): knife
obj1 (3682): spoon
obj1 (7425): bowl
obj1 (2346): banana
obj1 (1662): apple
ob

In [7]:
print(len(set(total)))
print(len(nms))

122218
80


In [15]:
#save data into force layout
graph_data = {
    "nodes": [],
    "links": []
}
for i, obj1 in enumerate(object_net):
    graph_data["nodes"].append(
        {
            "id": obj1,
            "group": 1
        }
    )
    for j, obj2 in enumerate(object_net[obj1]):
        if obj1 != obj2:
            graph_data["links"].append(
                {
                    "source": i,
                    "target": j,
                    #"value": object_net[obj1][obj2]
                    "weight": object_net[obj1][obj2]
                }
            )
        
with open('outputs/social-centric/object_graph.json', 'w') as json_file:
  json.dump(graph_data, json_file)

Ego-centric Constellation

In [9]:
object_net["toothbrush"]

{'person': 0.008367268079541064,
 'bicycle': 0.0009013068949977468,
 'car': 0.0007237461098646595,
 'motorcycle': 0.0,
 'airplane': 0.0,
 'bus': 0.0,
 'train': 0.0,
 'truck': 0.00013482540110556829,
 'boat': 0.0007170172084130019,
 'traffic light': 0.00018621973929236498,
 'fire hydrant': 0.0,
 'stop sign': 0.0,
 'parking meter': 0.0,
 'bench': 0.000730887297178775,
 'bird': 0.001364566750056857,
 'cat': 0.008309726156751653,
 'dog': 0.0034025787965616047,
 'horse': 0.0004868549172346641,
 'sheep': 0.0,
 'cow': 0.0,
 'elephant': 0.0,
 'bear': 0.0,
 'zebra': 0.0,
 'giraffe': 0.0,
 'backpack': 0.0011783767859773163,
 'umbrella': 0.0011589723778249951,
 'handbag': 0.0040535560741923595,
 'tie': 0.0010018032458425166,
 'suitcase': 0.0005640157924421884,
 'frisbee': 0.0,
 'skis': 0.00023573785950023574,
 'snowboard': 0.0,
 'sports ball': 0.00036563071297989033,
 'kite': 0.0,
 'baseball bat': 0.0,
 'baseball glove': 0.0,
 'skateboard': 0.0,
 'surfboard': 0.0,
 'tennis racket': 0.000217344055

In [10]:
# Spreading Activiation
decay = 0.8
energy = 1.0
visited = []
visited_full = []
parent = ""
def spreading(node_list, visited, visited_full, energy_threshold = 0.07):
#     print("expending..")
#     print(node_list)
#     print("---- visited ----- ")
#     print(visited)
    for obj_item in node_list:
        parent = obj_item[0]
        obj = obj_item[1]
        depth = obj_item[2]
        energy = obj_item[3]
#         print("energy: %f" % energy)
#         print("d: %i" % depth)
#         print("parent: %s" % parent)
        if obj not in object_net:
            continue
            
#         print(">>> %s" % obj)
        if parent == "":
            score = 1.0
        else:
            score = obj_item[4]#object_net[parent][obj]*energy
            
        node_list.remove(obj_item)
        
        visited.append(obj)
        visited_full.append((parent, obj, depth, energy, score, energy_threshold))
        
        friend_list = [(obj, i, depth+1, energy*decay, object_net[obj][i]*energy, energy_threshold) for i in object_net[obj] if object_net[obj][i]*energy > energy_threshold]
        a = [i for i in friend_list if i[1] not in visited]
        node_list.extend(a)
#         print(node_list)
#         print("-----------------------")
        if len(node_list) > 0 and depth <= 6:
            spreading(node_list, visited, visited_full, energy_threshold)
        else:
#             print("return %s, depth: %d" % (obj, depth))
            return []
    
    return visited_full
            

In [11]:
group = spreading([("", "chair", 0, 1.0)], [], [], 0.08)

In [12]:
print(len(group))
group2 = [i[1] for i in group]
print(len(set(group2)))

94
31


In [13]:
import networkx as nx
from networkx.readwrite import json_graph

In [16]:
specific_objects = ["person", "chair", "cell phone", "toaster", "bottle", "oven", "toilet", "refrigerator", "clock", "microwave"]

for s_obj in specific_objects:
    G = nx.DiGraph()
    visited=[]    
    group = spreading([("", s_obj, 0, 1.0)], [], [], 0.04)
    group2 = [i[1] for i in group]
    
#     print("%s: %d -> %d -> %d" % (s_obj, len(object_net[s_obj]), len(group), len(set(group2))))
    
    if len(set(group2)) == 0:
        continue
    
    root=""
    for i, obj in enumerate(group):
        if obj[1] in visited:
            continue
        
        if obj[0] == "":
            root = obj[1]
        else:
            if obj[1] not in visited:
                distance = obj[2]
                score = obj[4]
                
                print("%s-%s: (w=%f, score=%f, energy=%f, t=%f), %d" % (obj[0], obj[1], object_net[obj[0]][obj[1]], score, energy,obj[5], distance))
                G.add_edge(obj[0], obj[1], weight=object_net[obj[0]][obj[1]])
        
        visited.append(obj[1])
    
    print("----------------------")
    tree_data = json_graph.tree_data(G, root=root, attrs={'children': 'children', 'id': 'id'})
    H = json_graph.tree_graph(tree_data)
    
    with open('outputs/ego-centric/object_tree_coco_%s.json' % s_obj, 'w') as json_file:
        json.dump(tree_data, json_file)    

person-bicycle: (w=0.040997, score=0.040997, energy=1.000000, t=0.040000), 1
person-car: (w=0.125544, score=0.125544, energy=1.000000, t=0.040000), 1
person-motorcycle: (w=0.043135, score=0.043135, energy=1.000000, t=0.040000), 1
person-bus: (w=0.046523, score=0.046523, energy=1.000000, t=0.040000), 1
person-truck: (w=0.060130, score=0.060130, energy=1.000000, t=0.040000), 1
person-bench: (w=0.062526, score=0.062526, energy=1.000000, t=0.040000), 1
person-backpack: (w=0.077961, score=0.077961, energy=1.000000, t=0.040000), 1
person-umbrella: (w=0.053343, score=0.053343, energy=1.000000, t=0.040000), 1
person-handbag: (w=0.096429, score=0.096429, energy=1.000000, t=0.040000), 1
person-tie: (w=0.056275, score=0.056275, energy=1.000000, t=0.040000), 1
person-skis: (w=0.047333, score=0.047333, energy=1.000000, t=0.040000), 1
person-sports ball: (w=0.063539, score=0.063539, energy=1.000000, t=0.040000), 1
person-baseball glove: (w=0.040412, score=0.040412, energy=1.000000, t=0.040000), 1
pe